In [10]:
# script to save the rankings for the mechanism
import os
import h5py
import sys
import copy
import glob
import pickle
import pandas as pd

import numpy as np
import rmgpy.chemkin
import cantera as ct

import matplotlib.pyplot as plt
%matplotlib inline

# sys.path.append('/work/westgroup/harris.se/autoscience/reaction_calculator/dft')
# import importlib
# import thermokinetic_fun
# importlib.reload(thermokinetic_fun)

In [3]:
basedir = '/work/westgroup/harris.se/autoscience/reaction_calculator/delay_uncertainty/base_rmg_1week'

cantera_file = os.path.join(basedir, 'chem_annotated.cti')
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport, use_chemkin_names=True)

gas = ct.Solution(cantera_file)
perturbed_cti_path = os.path.join(basedir, 'perturbed.cti')
perturbed_gas = ct.Solution(perturbed_cti_path)

# This cti -> rmg converter dictionary can be made using rmg_tools/ct2rmg_dict.py
with open(os.path.join(basedir, 'ct2rmg_rxn.pickle'), 'rb') as handle:
    ct2rmg_rxn = pickle.load(handle)

print(f'{len(species_list)} species loaded')
print(f'{len(reaction_list)} reactions loaded')

130 species loaded
2488 reactions loaded


In [4]:
# rxn_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/reaction_uncertainty.npy'
# sp_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/species_uncertainty.npy'

rxn_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_reaction_uncertainty.npy'
sp_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_species_uncertainty.npy'

rmg_rxn_uncertainty = np.load(rxn_uncertainty_file)
rmg_sp_uncertainty = np.load(sp_uncertainty_file)

assert len(rmg_rxn_uncertainty) == len(reaction_list)
assert len(rmg_sp_uncertainty) == len(species_list)


rxn_uncertainty = np.zeros(len(gas.reactions()))
for ct_index in range(len(rxn_uncertainty)):
    rxn_uncertainty[ct_index] = rmg_rxn_uncertainty[ct2rmg_rxn[ct_index]]

# Cantera species should be in same rmg order, but this makes sure for us
for i in range(len(species_list)):
    assert str(species_list[i]) == gas.species_names[i]

sp_uncertainty = rmg_sp_uncertainty

total_uncertainty_array = np.concatenate((sp_uncertainty, rxn_uncertainty), axis=0)
total_uncertainty_mat = np.repeat(np.transpose(np.matrix(total_uncertainty_array)), 12 * 51, axis=1)


In [29]:
# Gather flame speed files
p_index = 0
working_dir = '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week'
fs_files = glob.glob(os.path.join(working_dir, 'coarse', 'flame_speeds*.npy'))
# fs_files = glob.glob(os.path.join(working_dir, f'param_{p_index:04}', 'perturbed_flame_tight*.h5'))

In [33]:
len(gas.reactions())

2523

In [32]:
len(np.load(fs_files[0]))

2523

In [30]:
fs_files

['/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/coarse/flame_speeds_0000.npy',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/coarse/flame_speeds_0050.npy',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/coarse/flame_speeds_0100.npy',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/coarse/flame_speeds_0150.npy',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/coarse/flame_speeds_0200.npy',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/coarse/flame_speeds_0250.npy',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/coarse/flame_speeds_0300.npy',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/coarse/flame_speeds_0350.npy',
 '/work/westgroup/harris.se/auto

In [11]:
# f = h5py.File(fs_files[0], 'r')

In [12]:
arr2 = ct.SolutionArray(gas)
arr2.read_hdf(fs_files[0], group="freeflame", subgroup="flame", force=True)

{'Sim1D_type': 'FreeFlame',
 'cantera_version': '2.5.1',
 'curve': 0.01,
 'date': 'Sun, 14 May 2023 07:55:06 -0400',
 'description': 'butane flame',
 'energy_enabled': True,
 'fixed_temperature': 651.7678282025165,
 'git_commit': 'b0bace782',
 'max_grid_points': 30000,
 'max_time_step_count': 1000,
 'prune': 0.001,
 'radiation_enabled': False,
 'ratio': 2.0,
 'slope': 0.01,
 'soret_enabled': False,
 'transport_model': 'Mix'}

In [24]:
width = 0.08
flame = ct.FreeFlame(gas, width=width)
flame.set_initial_guess(data=arr2)

In [28]:
flame.velocity

array([0.1196952 , 0.1196952 , 0.1196952 , ..., 0.7014142 , 0.70141468,
       0.70141468])

In [27]:
dir(flame)


['L',
 'P',
 'T',
 'V',
 'X',
 'Y',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_get_indices',
 '_get_initial_solution',
 '_interrupt',
 '_other',
 '_steady_callback',
 '_time_step_callback',
 'boundary_emissivities',
 'chemical_potentials',
 'clear_stats',
 'collect_data',
 'concentrations',
 'cp',
 'cp_mass',
 'cp_mole',
 'creation_rates',
 'cv',
 'cv_mass',
 'cv_mole',
 'delta_enthalpy',
 'delta_entropy',
 'delta_gibbs',
 'delta_standard_enthalpy',
 'delta_standard_entropy',
 'delta_standard_gibbs',
 'density',
 'density_mass',
 'density_mole',
 'destruction_rates',
 'domain_index',
 'domains',
 'electrochemical_potentials',
 'elemental_mass_fraction',
 'elem

In [19]:
arr2.s

array([6859.01804931, 6859.01804931, 6859.01804961, ..., 9078.55049703,
       9078.55111037, 9078.55111037])

In [23]:
arr2.u

array([-163880.13380868, -163880.13383085, -163880.13383324, ...,
       -576106.80503064, -576106.25921701, -576106.2592594 ])

In [20]:
arr2.volume_mass

array([0.83711215, 0.83711215, 0.83711215, ..., 4.90547968, 4.905483  ,
       4.905483  ])

In [21]:
arr2.volume_mole

array([ 24.61720982,  24.61720982,  24.61720982, ..., 140.23056899,
       140.23066727, 140.23066727])

In [16]:
dir(arr2)

['DP',
 'DPQ',
 'DPX',
 'DPY',
 'HP',
 'HPQ',
 'HPX',
 'HPY',
 'ID',
 'P',
 'PQ',
 'PV',
 'PX',
 'P_sat',
 'Q',
 'SH',
 'SP',
 'SPQ',
 'SPX',
 'SPY',
 'ST',
 'SV',
 'SVQ',
 'SVX',
 'SVY',
 'T',
 'TD',
 'TDQ',
 'TDX',
 'TDY',
 'TH',
 'TP',
 'TPQ',
 'TPX',
 'TPY',
 'TQ',
 'TV',
 'TX',
 'T_sat',
 'UP',
 'UV',
 'UVQ',
 'UVX',
 'UVY',
 'VH',
 'X',
 'Y',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_scalar',
 '_extra',
 '_indices',
 '_interface_n_species',
 '_interface_passthrough',
 '_meta',
 '_n_reactions',
 '_n_species',
 '_n_species2',
 '_n_total_species',
 '_output_dummy',
 '_passthrough',
 '_phase',
 '_purefluid_sca

In [8]:
fs_files

['/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/param_0000/perturbed_flame_tight_0.h5',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/param_0000/perturbed_flame_tight_1.h5',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/param_0000/perturbed_flame_tight_10.h5',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/param_0000/perturbed_flame_tight_11.h5',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/param_0000/perturbed_flame_tight_12.h5',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/param_0000/perturbed_flame_tight_13.h5',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1week/param_0000/perturbed_flame_tight_14.h5',
 '/work/westgroup/harris.se/autoscience/reaction_calculator/fs_sensitivity/base_rmg_1